# Implicit Lambdas (The Power of Comma)

The Rexl language shares some heritage and design with Power Fx, the language of Power Apps.

Both grew from an effort circa 2010 to design a general expression language that would be suitable for
interactive data exploration in a variety of host applications.

The Excel formula language is the most broadly used expression language on planet Earth. While broadly used,
it is tightly coupled to _grid addressing_ in a spread sheet. We operate with _named values_ instead.

## Table

The Excel language does not directly support tables with values arranged in named columns.
Instead it supports cell ranges. We directly support both record (a single row from a table)
and table (sequence of record). A table column in Rexl is a natural analog of a range of cells
in Excel.

We first import some customer order data to use for illustrations.

In [1]:
import "../data/Orders.rexl";

In [2]:
#!globals

Variable,Type
Orders,"{Cid:s, Pid:s, Price:r8, Quantity:i8}*"


The `Orders` table has the columns:
* `Cid`: customer id of type text.
* `Pid`: product id of type text.
* `Price`: the price per item.
* `Quantity`: the number of items.

In [3]:
Orders

Seq<{str,str,r8,i8}>
   0) { Cid: C01, Pid: P01, Price: 2.5, Quantity: 3 }
   1) { Cid: C01, Pid: P02, Price: 8, Quantity: 2 }
   2) { Cid: C02, Pid: P01, Price: 2.25, Quantity: 7 }
   3) { Cid: C01, Pid: P01, Price: 2.5, Quantity: 5 }
   4) { Cid: C03, Pid: P02, Price: 7.5, Quantity: 5 }
   5) { Cid: C04, Pid: P01, Price: 3, Quantity: 1 }


## Sum

Excel has a `SUM` function that adds the contents of the cells in a range. We also need a `Sum` function
that takes a table column. The notation `Orders.Quantity` is a natural way to refer to the `Quantity`
column of the `Orders` table. For example, to compute the total quantity of items represented in the
`Orders` table, use:

In [4]:
Sum(Orders.Quantity)

23


Excel also has a `SUMPRODUCT` function that computes the product of corresponding items in two parallel
cell ranges and adds all of those products. If we had a `SumProduct` function, perhaps one could
write `SumProduct(Orders.Quantity, Orders.Price)` to get the total value of all orders. We really didn't
want to pursue this route since the table name `Orders` is repeated and this approach doesn't generalize
to operations other than _multiplying_ the two column values.

After puzzling over this for a while, we realized that we should replace the dot with a comma. That is,
instead of `Sum(Orders.Quantity)`, we should encourage:

In [5]:
Sum(Orders, Quantity)

23


What is going on here? And how does it help avoid `SumProduct` and its limitations?
Read on.

## Lambdas

Many languages have a mechanism for writing an _inline function_ that can be passed as a
function argument. Such an inline function notation is often called a lambda expression.
For example, in C#, the sum would be written
```
Enumerable.Sum(Orders, order => order.Quantity)
```
Replacing dot with comma is effectively introducing a lambda expression but without
the need to name the sequence item (`order`).

Rexl does _allow_ the sequence item to be named, when needed, but within the first argument, not
within the implicit lambda expression:

In [6]:
Sum(order: Orders, order.Quantity)

23


The name of the `order` record can also be specified using `as`:

In [7]:
Sum(Orders as order, order.Quantity)

23


Rexl also allows using the `it` keyword to reference the sequence item:

In [8]:
Sum(Orders, it.Quantity)

23


## SumProduct

With comma instead of dot and implicit lambdas, there is no need for a separate `SumProduct` function.
Instead, one simply writes:

In [9]:
Sum(Orders, Quantity * Price)

92.25


As before, one can introduce a name for the order row, for example:

In [10]:
Sum(row: Orders, row.Quantity * row.Price)

92.25


This generalizes to arbitrary logic. For example, to compute the _revenue reduction_
if we give customer `C01` an 8% discount, use:

In [11]:
Sum(Orders, Quantity * Price * If(Cid = "C01", 8%, 0%))

2.88


In Excel, this would typically be done by first adding a new column containing the discount
formula, then by applying `SUM` to the cell range of that column.

## Other Functions

Many Rexl functions support implicit lambda arguments.

To filter `Orders` to only rows where `Quantity` is at least `4`, use the `TakeIf` function,
where the 2nd argument of `TakeIf` is an implicit lambda:

In [12]:
TakeIf(Orders, Quantity >= 4)

Seq<{str,str,r8,i8}>
   0) { Cid: C02, Pid: P01, Price: 2.25, Quantity: 7 }
   1) { Cid: C01, Pid: P01, Price: 2.5, Quantity: 5 }
   2) { Cid: C03, Pid: P02, Price: 7.5, Quantity: 5 }


To add a `Cost` column that contains the product of `Quantity` and `Price`, use the `ForEach`
and `SetFields` functions, where the 2nd argument of `ForEach` is an implicit lambda:

In [13]:
ForEach(row: Orders, SetFields(row, Cost: Quantity * Price))

Seq<{str,r8,str,r8,i8}>
   0) { Cid: C01, Cost: 7.5, Pid: P01, Price: 2.5, Quantity: 3 }
   1) { Cid: C01, Cost: 16, Pid: P02, Price: 8, Quantity: 2 }
   2) { Cid: C02, Cost: 15.75, Pid: P01, Price: 2.25, Quantity: 7 }
   3) { Cid: C01, Cost: 12.5, Pid: P01, Price: 2.5, Quantity: 5 }
   4) { Cid: C03, Cost: 37.5, Pid: P02, Price: 7.5, Quantity: 5 }
   5) { Cid: C04, Cost: 3, Pid: P01, Price: 3, Quantity: 1 }


To compute `100` factorial, use the `Fold` and `Sequence` functions, where the 3rd
argument of `Fold` is an implicit lambda:

In [14]:
Fold(k: Sequence(100), v: 1ia, v * k)

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000


Final note: unlike Excel, we chose to make function names use capital letters at the beginning
of each word, as in `Sum`, `ForEach`, and `TakeIf`, not all characters of the function name.